In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf


In [4]:
df = pd.read_csv(r"/Users/priyankapalaniselvam/Downloads/classifier_one_data/full_df.csv")
if 'ID' in df.columns:
    df = df.drop('ID', axis=1)

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['labels'])
df['labels'] = df['labels'].astype(str)
left_images = df[['Left-Fundus', 'labels']]
right_images = df[['Right-Fundus', 'labels']]

left_images.columns = ['image', 'target']
right_images.columns = ['image', 'target']

combined_df = pd.concat([left_images, right_images])
combined_df = combined_df.dropna(subset=['image'])

train_df, test_df = train_test_split(combined_df, test_size=0.2, random_state=42)

In [5]:
datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='/Users/priyankapalaniselvam/Downloads/classifier_one_data/ODIR-5K/ODIR-5K/Training Images',
    x_col='image',
    y_col='target',
    class_mode="categorical",
    target_size=(224, 224),
    batch_size=16
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='/Users/priyankapalaniselvam/Downloads/classifier_one_data/ODIR-5K/ODIR-5K/Training Images',
    x_col="image",
    y_col="target",
    class_mode="categorical",
    target_size=(224, 224),
    batch_size=32
)

Found 10227 validated image filenames belonging to 8 classes.
Found 2557 validated image filenames belonging to 8 classes.


In [6]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))

model.add(Dense(8, activation='softmax'))

/opt/anaconda3/envs/test310/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
)

Epoch 1/20


/opt/anaconda3/envs/test310/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


640/640 ━━━━━━━━━━━━━━━━━━━━ 828s 1s/step - accuracy: 0.4193 - loss: 1.7211 - val_accuracy: 0.4509 - val_loss: 1.5731
Epoch 2/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 862s 1s/step - accuracy: 0.4444 - loss: 1.5922 - val_accuracy: 0.4462 - val_loss: 1.5746
Epoch 3/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 760s 1s/step - accuracy: 0.4469 - loss: 1.5606 - val_accuracy: 0.4478 - val_loss: 1.5430
Epoch 4/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 763s 1s/step - accuracy: 0.4474 - loss: 1.5638 - val_accuracy: 0.4501 - val_loss: 1.5309
Epoch 5/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 842s 1s/step - accuracy: 0.4459 - loss: 1.5413 - val_accuracy: 0.4533 - val_loss: 1.4928
Epoch 6/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 1018s 2s/step - accuracy: 0.4477 - loss: 1.5115 - val_accuracy: 0.4564 - val_loss: 1.4755
Epoch 7/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2258s 4s/step - accuracy: 0.4604 - loss: 1.4696 - val_accuracy: 0.4447 - val_loss: 1.4422
Epoch 8/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 10348s 16s/step - accuracy: 0.4594 - loss: 1.4573 - val_accuracy: 

2024-10-29 01:17:17.457921: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 4 of 8
2024-10-29 01:17:19.819674: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


640/640 ━━━━━━━━━━━━━━━━━━━━ 1245s 1s/step - accuracy: 0.4611 - loss: 1.4117 - val_accuracy: 0.4638 - val_loss: 1.3756
Epoch 11/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 1278s 2s/step - accuracy: 0.4764 - loss: 1.3747 - val_accuracy: 0.4650 - val_loss: 1.3590
Epoch 12/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 1280s 2s/step - accuracy: 0.4675 - loss: 1.3749 - val_accuracy: 0.4642 - val_loss: 1.3360
Epoch 13/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 1169s 2s/step - accuracy: 0.4755 - loss: 1.3438 - val_accuracy: 0.4806 - val_loss: 1.3210
Epoch 14/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 1154s 2s/step - accuracy: 0.4800 - loss: 1.3341 - val_accuracy: 0.4955 - val_loss: 1.3145
Epoch 15/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 1134s 2s/step - accuracy: 0.4815 - loss: 1.3281 - val_accuracy: 0.4885 - val_loss: 1.3039
Epoch 16/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 1092s 2s/step - accuracy: 0.4926 - loss: 1.2971 - val_accuracy: 0.5025 - val_loss: 1.2976
Epoch 17/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 1109s 2s/step - accuracy: 0.4924 - loss: 1.2614 - val_

In [12]:
model.save_weights('base_model_cn.weights.h5')
